In [24]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import os
import numpy as np
import _pickle as cpickle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [27]:
def pickle_chr(pickle_filename, db_filename, chunksize):
    # remove previous pickle file if exists
    if os.path.exists(pickle_filename): os.remove(pickle_filename) 
    # wb+ to ensure previous pkl files of same name arent appended to
    with open(pickle_filename,'wb+') as file:
        for chunk in pd.read_csv(db_filename, engine='c', sep='\t', usecols=[0], index_col=False, na_values=None, dtype={'#CHROM:':'string'}, chunksize=chunksize): 
            cpickle.dump(chunk, file)
# {"#CHROM":"category"}
# open pickle file --> for each chunk in database's chr column: dump the chunk into pickle

In [28]:
def pickle_val(pickle_filename, db_filename, val_cols, dtypes, chunksize):
    if os.path.exists(pickle_filename): os.remove(pickle_filename) 
    with open(pick_filename, 'wb+') as file:
        for chunk in pd.read_csv(db_filename, engine='c', sep='\t', usecols=valcols, index_col=False, dtype=dtypes, chunksize=chunksize): 
            cpickle.dump(chunk,file)

In [ ]:
# def chr_idx(database, chr_raw, chunksize=10**4):
# chr_raw True == chr1...chrY, chr_raw false == 1...Y
# database = 'gene4denovo201907'
database = "gnomad211_exome"
chunksize = 10 ** 4

pickle_chr('%s.pkl' % database, 'databases/hg38_%s.txt' % database, chunksize)

chromosome_list = [x+1 for x in range(22)] + ["X", "Y"]
# ["chr" + str(x+1) for x in range(22)] + ["chrX", "chrY"]  

total_chr = []

In [ ]:
with open("%s.pkl" % database, 'rb') as file:
    curr_chr = 0 # counts until last index of chromosome list
    chunk_counter = 0 # multiply by chunk size to get actual - not relative - index
    while True:
        try:
            result = cpickle.load(file)
            chunk = pd.Series(dtype="category")
            chunk = result
            del result
        # EOFError when last chunk has size < chunk size
        except EOFError:
            break
        else:
            # if last value of chunk is next chr, find where index of next chr is
            curr_chunk_idx = chunksize * chunk_counter
            try:
                next_chr = chromosome_list[curr_chr + 1]
            except IndexError:
                ...
            if chunk.iloc[-1][0] == next_chr:
                print(chunk.describe())#.iloc[3][0])) # + curr_chunk_idx))
                curr_chr += 1
            chunk_counter += 1
            del chunk
#x = chr_idx("gnomad211_exome", chr_raw=True, chunksize=10 ** 5)

In [ ]:
# splitter = np.random.choice([0, 1, 2], 10 ** 4)

with open("gnomad211_exome.pkl", 'rb') as file:
    # load file up until x, y
    result = cpickle.load(file)
    chromosome_list = [str(x+1) for x in range(22)] + ["X", "Y"]
    chunk = pd.Series(pd.Categorical.from_codes(result, categories=chromosome_list))
chunk

In [ ]:
with open("exac03_chr1.pkl", 'wb+') as file:
    for chunk in pd.read_csv("databases/hg38_exac03.txt", nrows=1089537,engine='c', sep='\t', usecols=[1,5,6,7,8,9,10,11,12], index_col="Start", dtype={"Start":"int","ExAC_ALL":"float","ExAC_AFR":"float","ExAC_AMR":"float","ExAC_EAS":"float","ExAC_FIN":"float","ExAC_NFE":"float","ExAC_OTH":"float","ExAC_SAS":"float"}, na_values=".", chunksize=10**4): 
        cpickle.dump(chunk,file)

In [80]:
test = pd.read_csv("databases/hg38_gene4denovo201907.txt", sep='\t', usecols=[1,5,6,7,8,9,10], index_col="POS", dtype={"POS":"int"})

In [81]:
test.iloc[15208]

DN ID                                 dn326360
Patient ID                               14458
Phenotype                                  ASD
Platform                                   WGS
Study         Joon-Yong An et al. Science 2018
Pubmed ID                             30545852
Name: 71120533, dtype: object